<a href="https://colab.research.google.com/github/akalakheti/Heatmap/blob/master/Heatmap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tutorial for visualizing activation heatmaps**

# 1. Import all the libraries required

In [0]:
import cv2 #For image processing
import numpy as np # for math operation
import tensorflow as tf # main framework

# Load Image for processing with the size 224x224
(You could use image with any size but the model we will be using requires an image with size 224 x 224 )

In [0]:
IMAGE_PATH = 'bee.jpg'
image = tf.keras.preprocessing.image.load_img(IMAGE_PATH, target_size=(224, 224))
image = tf.keras.preprocessing.image.img_to_array(image)

# Download a pre-trained classification model or create your own
(For this example, we will be using pre-trained vgg16 model)

In [0]:
model = tf.keras.applications.VGG16(weights='imagenet', include_top=True)

conv_layer = model.get_layer("block5_conv3") #Get the last convolation layer

CAM_model = tf.keras.models.Model([model.inputs], [conv_layer.output, model.output]) #Create a model that outputs target convolution and output


CLASS_INDEX = 309 #Index of the class you want to classify in the output layer of model

553467904/553467096 [==============================] - 17s 0us/step


# Find the score for target class

In [0]:

with tf.GradientTape() as tape:
    otpts, predictions = CAM_model(np.array([image]))
    loss = predictions[:, CLASS_INDEX]


# Get the filters and gradients from above calculation

In [0]:
output = otpts[0]
grads = tape.gradient(loss, otpts)[0]

In [0]:
weights = tf.reduce_mean(grads, axis=(0, 1)) #Average the gradient obtained.

# Create a Numpy array according to gradient importance and calculate weighted output

In [0]:
weighted_cam = np.ones(output.shape[0:2], dtype=np.float32)

In [0]:
for i, w in enumerate(weights):
    weighted_cam += w * output[:, :, i]


# Use the weighted numpy array along with cv2 library to convert it into an image

In [0]:
weighted_cam = cv2.resize(weighted_cam.numpy(), (224, 224))
weighted_cam = np.maximum(weighted_cam, 0)
heatmap = (weighted_cam - weighted_cam.min()) / (weighted_cam.max() - weighted_cam.min())

In [0]:
weighted_cam = cv2.applyColorMap(np.uint8(255*heatmap), cv2.COLORMAP_JET) # create the heatmap image
output_image = cv2.addWeighted(cv2.cvtColor(image.astype('uint8'), cv2.COLOR_RGB2BGR), 0.7, weighted_cam, 1, 0) #overlapping the heatmap with original image 

# Finally, save the image.

In [0]:
cv2.imwrite('cam2.png', output_image)

True

Output image: https://imgur.com/I5VPtFs